In [42]:
import csv

def to_dict(columns, data):
    d = {}
    for ind, category in enumerate(columns):
        d[category] = data[ind]
    return d

key_songs = {}

def update_key_songs(title, values):
    if title not in key_songs:
        key_songs[title] = {}
    for key in values:
        key_songs[title][key] = values[key]

In [43]:
# https://www.kaggle.com/edumucelli/spotifys-worldwide-daily-song-ranking
regions = set()

with open("num_streams.csv") as f:
    reader = csv.reader(f)
    columns = next(reader)
    print(columns)
    for ind, line in enumerate(reader):
        if ind > 0:
            line_dict = to_dict(columns, line)
            regions.add(line_dict['Region'])
            if line_dict['Region'] in ['us', 'ca', 'mx']:
                update_key_songs(line_dict['Track Name'].lower(), line_dict)
                count += 1

print(count, len(key_songs.keys()))

['Position', 'Track Name', 'Artist', 'Streams', 'URL', 'Date', 'Region']
7924216 2442


In [44]:
# https://www.kaggle.com/mrmorj/dataset-of-songs-in-spotify?select=genres_v2.csv
intersection = set()

with open("genres_v2.csv") as f:
    reader = csv.reader(f)
    columns = next(reader)
    print(columns)
    for ind, line in enumerate(reader):
        if ind > 0:
            line_dict = to_dict(columns, line)
            if line_dict['song_name'].lower() in key_songs:
                update_key_songs(line_dict['song_name'].lower(), line_dict)
                intersection.add(line_dict['song_name'].lower())

updated = {}
for song in intersection:
    updated[song] = key_songs[song]
key_songs = updated
print(len(intersection), len(key_songs))

['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature', 'genre', 'song_name', 'Unnamed: 0', 'title']
682 682


In [45]:
# https://www.kaggle.com/yamaerenay/spotify-dataset-19212020-160k-tracks?select=data.csv
intersection = set()

with open("all_tracks.csv") as f:
    reader = csv.reader(f)
    columns = next(reader)
    print(columns)
    for ind, line in enumerate(reader):
        if ind > 0:
            line_dict = to_dict(columns, line)
            if line_dict['name'].lower() in key_songs:
                update_key_songs(line_dict['name'].lower(), line_dict)
                intersection.add(line_dict['name'].lower())

updated = {}
for song in intersection:
    updated[song] = key_songs[song]
key_songs = updated
print(len(intersection), len(key_songs))

['acousticness', 'artists', 'danceability', 'duration_ms', 'energy', 'explicit', 'id', 'instrumentalness', 'key', 'liveness', 'loudness', 'mode', 'name', 'popularity', 'release_date', 'speechiness', 'tempo', 'valence', 'year']
682
682 549
549 549


In [73]:

categories = [category for category in key_songs[list(key_songs.keys())[0]]]

print(len(categories))
print(categories)
for category in ["URL","Date","Region","id","uri",
                 "track_href","analysis_url","song_name",
                 "Unnamed: 0","title","type","name", "artists",
                "release_date"]:
    if category in categories:
        categories.remove(category)
print(len(categories))
print(categories)

category_counter = {}
category_type = {}

def is_numeral(s):
    try:
        _ = float(s)
        return True
    except ValueError:
        return False

dummies = {
    "mode": ["minor", "major"],
    "explicit": ["not explicit", "explicit"],
     "key": ["C", "C♯", "D", "D♯", "E", "F", "F♯", "G", "G♯", "A", "A♯", "B"]
}

for category in categories:
    category_counter[category] = set()
    category_type[category] = "NUMERICAL"
    
for song in key_songs:
    if song == 'From The D To The A (feat. Lil Yachty)'.lower():
        continue
    for category in categories:
        category_counter[category].add(key_songs[song][category])
        if not is_numeral(key_songs[song][category]):
            category_type[category] = "CATEGORICAL"
        if category in dummies and key_songs[song][category].isnumeric():
            key_songs[song][category] = dummies[category][int(key_songs[song][category])]
        

category_type["key"] = "CATEGORICAL"

for key in category_counter:
    print(key, len(category_counter[key]), category_type[key])
    if category_type[key] == "CATEGORICAL":
        category_type[key] = lambda x: ("'{}'" if is_numeral(x) else '"{}"').format(x.replace('"', '""'))
    else:
        category_type[key] = lambda x: x
        

with open("code/hw1/public/spotify.csv", "w") as f:
    f.write(",".join([c.lower() for c in categories])+"\n")
    for song in key_songs:
        f.write(",".join([category_type[category](key_songs[song][category]) for category in categories]) + "\n")

35
['Position', 'Track Name', 'Artist', 'Streams', 'URL', 'Date', 'Region', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'type', 'id', 'uri', 'track_href', 'analysis_url', 'duration_ms', 'time_signature', 'genre', 'song_name', 'Unnamed: 0', 'title', 'artists', 'explicit', 'name', 'popularity', 'release_date', 'year']
21
['Position', 'Track Name', 'Artist', 'Streams', 'danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', 'duration_ms', 'time_signature', 'genre', 'explicit', 'popularity', 'year']
Position 138 NUMERICAL
Track Name 548 CATEGORICAL
Artist 249 CATEGORICAL
Streams 546 NUMERICAL
danceability 352 NUMERICAL
energy 391 NUMERICAL
key 12 CATEGORICAL
loudness 531 NUMERICAL
mode 2 CATEGORICAL
speechiness 408 NUMERICAL
acousticness 486 NUMERICAL
instrumentalness 265 NUMERICAL
liveness 346 NUMERICAL
valence 394 NUME